# Supplemental notes, getting started

We will be using the same tools that we used last week for this session.

- [pandas](pandas.pydata.org) for data handling (our dataframe library)
- [seaborn](seaborn.pydata.org) for _nice_ data visualization

We will also be using:

- [scikit-learn](scikit-learn.org) an extensive machine learning library.
- [numpy](numpy.org) - a fundamental maths library.  We won't explore it much today, but it does have some useful methods that we'll need.  It underlies all other mathematical and plotting tools that we use in Python.

We'll be using scikit-learn and it's well worth reading the documentation and high level descriptions.

A useful resource is Stack Overflow - if you have a question that sounds like 'how do I do {x}' then someone will probably have answered it on SO. Questions are also tagged by library so if you have a particular pandas question you can do something like going to https://stackoverflow.com/questions/tagged/pandas (just replace the 'pandas' in the URL with whatever library you're trying to use.

Generally answers on SO are probably a lot closer to getting you up and running than the documentation. Once you get used to the library then the documentation is generally a quicker reference. We will cover strategies for getting help in class.


## Git links

If you are just starting out using git, or are interested in learning here are some references that may be useful.

- GitKraken interface basics: https://support.gitkraken.com/start-here/interface
- Staging and committing (save current state -> local history): https://support.gitkraken.com/working-with-commits/commits
- Pushing and pulling (sync local history <-> GitHub history): https://support.gitkraken.com/working-with-repositories/pushing-and-pulling
- Forking and pull requests (request to sync your GitHub history <-> someone else's history - requires a _review_):
  - https://help.github.com/articles/about-forks/
  - https://help.github.com/articles/creating-a-pull-request-from-a-fork/

## Recap of last time

Last week we used Scikit-Learn and Seaborn to do least-squares linear regression.  This is the same kind of linear regression that you've applied countless times in Excel.  We reframed it in the context of machine learning by creating a target array with labels, a features matrix, and then we used the fit method to train the model and the predict method to try to predict labels on new observations.  

As was highlighted last week, the common linear regression approach aims to find a line that minimised the squared error between the predictions that result from that line and the observed labels in the target array.  The statistical consequences of this were discussed.  These included that the distribution of the errors that we see in our observed features is gaussian, also known as the normal distribution or the bell curve.

We ran through an example where we created perfect linear data and then added gaussian noise.  We used Seaborn to do a linear fit and show us the residuals.  In this case we saw residuals that looked scattered in a way that was consistent with gaussian error and that indicated a good fit.

When the errors are not gaussian this distorts the linear fit found, and makes it a poorer predictor than it would be in the gaussian case. Distorted linear models are a general consequence of non-gaussian errors in least squares linear regression.  The use of robust regression was also introduced as a way to lessen the impact of data outliers. 

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import pandas as pd

from collections import namedtuple

from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# regression_help is a python file in the same directory as this notebook
# various functions will be imported throughout the notebook so make sure this import works
from regression_help import recovery_sulphur_dataframe_with_outliers

# Exercise 1: Metal recovery vs % sulphur in feed

In this exercise, we are going to look at a typical minerals engeineering problem. The data are representative of what would be collected in laboratory batch floatation tests on samples taken from different parts of a base metal orebody.

In this exercise we're going to review the impact of outliers on linear models, and we're going to try different methods of linear regression that use other measures of line error that are less sensitive to this.

We're going to use some pre-written methods provided in the file week4_help.py, which you should find in the same directory as this notebook.  You'll need to import them.  Please review how we import methods from modules that we've written ourselves.

It's not important to understand how these methods work, just to be able to use them for the exercise.

Import metal_sulphur_recovery_with_outliers.  Then use:

```metal_sulphur_recovery_with_outliers?```

to read the docstring.

We'll use this to create Pandas dataframes with varying numbers of outliers present.

In [ ]:
# from regression_help import recovery_sulphur_dataframe_with_outliers
recovery_sulphur_dataframe_with_outliers?

This should have brought up the help for the recovery_sulphur_dataframe_with_outliers method.

Let's start with no outliers. If we call this with zero as the parameter then it won't return any outliers.  Let's do that, and then plot the data.

In [ ]:
# Use the function recovery_sulphur_dataframe_with_outliers to get data without outliers
recovery = 

In [ ]:
recovery.head()

In [ ]:
sns.set(style="whitegrid")

In [ ]:
ax = plt.axes()
ax.plot(recovery['feed_sulphur_percent'], recovery['metal_recovery_percent'], 'o')
ax.set(xlim=(0, 1.5), ylim=(0, 100), xlabel='feed sulphur %', ylabel='recovery %',
      title='% metal recovery vs % sulphur in feed')

### Refresher from last time: Linear regression with Seaborn 
Now let's use Seaborn to try and fit a least squares linear regression to this plot.

The function we would like to try and use here is ```regplot```.

In [ ]:
# Plot metal recovery against feed sulphur


Note the confidence intervals in light blue that Seaborn has helpfully provided!  As you can see, confidence intervals are not very meaningful when the data doesn't conform with the assumptions of least-squares linear regression. There is a definite curve in the data and the line is unable to fit through all the points.

It is interesting that this model will predict greater than 100% recovery at feed sulphur beyond 2%, and ~40% metal recovery with no feed sulphur. Wouldn't that be nice! This may seem reasonable to a data scientist, but to a domain expert this is not possible.


Next, let's take a look at the residuals using ```residplot```.

In [ ]:
# Plot residuals


Looking at the residuals, they are also not gaussian - they have a structure to them.  You can see that for low and high values of feed_sulphur the residuals are negative, but they go positive for middle values.  It may be helpful to review the PowerPoint presentation on linear regression from last time, which highlights this as a behaviour that indicates the underlying assumptions of least squares linear regression aren't being met.

### Transform the data

Let's transform the data to make it more linear.  We can use the transform pipeline when using scikit-learn, but it's not available when using seaborn, so we'll do it in the more basic way that we used last week - we'll simply apply the transform to the feed_sulphur_percent.

What transform would be good for this data? (Hint: try reciprocal transformation, f(x) = 1/x)

Call it 'reciprocal_feed_sulphur' and store it in the ```recovery``` dataframe.

In [ ]:
recovery['reciprocal_feed_sulphur'] = 

In [ ]:
ax = plt.axes()
ax.plot(recovery['reciprocal_feed_sulphur'], recovery['metal_recovery_percent'], 'o')
ax.set(xlim=(0.6, 5), ylim=(0, 100), xlabel='1/feed sulphur %', ylabel='recovery %',
      title='% metal recovery vs % sulphur in feed')

That's better.  Now fitting a linear least squares line should be more sensible.  Repeat making the model using ```regplot```.

But what happens if we add outliers?  Outliers are an example of non-gaussian error and lead to poor least squares fits.  In practice, you can include them in a model if you understand the impact that they'll have on the regression, or you can take them out if why they are in the data is understood and it makes sense in the industry domain to ignore them.  But if they're being included in the model it often means that least squares will give you poor results and you should consider other line fitting methods.

Create the recovery data from with 0.2 probabability of outliers.

In [ ]:
recovery = 

In [ ]:
recovery['reciprocal_feed_sulphur'] = 1/recovery['feed_sulphur_percent']
sns.regplot(x='reciprocal_feed_sulphur', y='metal_recovery_percent', data=recovery)

Now that we've added a few outliers we see the terrible fit that only a few well placed outliers can cause.  These outliers are said to have "high leverage" because they are far away from the rest of the data, and, like having a long lever, they can move the earth (or at least your line).  Seaborn has a built in method for providing robust fitting.  Look in the help and try it out.

In [ ]:
sns.regplot?

In [ ]:
recovery['reciprocal_feed_sulphur'] = 1/recovery['feed_sulphur_percent']
sns.regplot(x='reciprocal_feed_sulphur', y='metal_recovery_percent', data=recovery, robust=True)

That's better.  Robust methods are based on ways of measuring central tendency other than the mean average.  Lets see what happens when I want to find an average weight and I have five people, with one being an outlier.

Weights: 72, 68, 76, 62 and 38 kilograms

If I take a mean average, it'd suggest that the average weight of peeople is about (38+62+68+72+76)/5 = 63kg.

Given the numbers that we have, this would seem a bit low.  This is analogous to how least-squares finds a mean average line through the data.

If instead we use a median average, which orders all of the numbers and picks the one in the middle (the 3rd, in a list of five numbers), then we get an average of 68kg.  This is closer to what seems intuitively correct, even with the 38kg outlier.  This is an example of how robust linear regression works, and in fact the method called Theil-Sen regression does exactly this - it finds the median line through the data.

Robust fitting methods are only resistant to outliers to a degree, and, depending on the specifics of the data and how many outliers are present, even robust methods fall over at some point.  This is often called the breakdown point of the method.  Sometimes there is underlying theory that can predict this, but often it's better to keep using exploratory visual tools to validate our models.

In [ ]:
# Increase the outliers parameter to 0.5 and fit a robust regression, remember to apply the transform


We can see that this is the breakdown point for Seaborn's robust (Theil-Sen) linear regression method. Let's try a scikit-learn robust regression.

### Refresher from last time: The Estimator API of scikit-learn

Now let's try this using scikit-learn. It has a greater range of robust methods available than Seaborn, and they are used in the same way that least squares linear regression is used.

Remember the sequence of steps for using scikit-learn for linear regression:

1. Choose a class of model by importing the appropriate estimator class.  In our case we want to import the least squares linear regression and robust regressors.  Here's how we can do it.

First import a robust regressor from scikit-learn

```from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor```

Now create an "instance" of your favorite robust regressor.  We can do it like this:

```
model = RANSACRegressor()
```

To check that this has worked look at the model object after it's created.  It should tell you about some of its settings.

```
model
```

In [ ]:
# from sklearn.linear_model import RANSACRegressor
model = RANSACRegressor()
model

2. Get the metal recovery dataframe with 20% outlier probability (0.2) and apply a reciprocal transform.  We then need to arrange a pandas dataframe into a features matrix, 'reciprocal_feed_sulphur', and a target vector, 'metal_recovery_percent'.

Remember the strange notation to get the the data into the right array shape.  Use two pairs of "[ ]" as "[[ ]]" for the features matrix.  These square brackets aren't needed in multivariate cases, they are only required for this univariate case, which is a simpler situation than scikit-learn is usually used for.

In [ ]:
recovery = recovery_sulphur_dataframe_with_outliers(0.2)
recovery['reciprocal_feed_sulphur'] = 1/recovery['feed_sulphur_percent']

features_matrix = 
target_array = 

3. Split the dataset into a training and test set using ```train_test_split``` and fit the model to your data by using the fit() method of the LinearRegression object. Remember to import the train_test_split function.

Look at the documentation for how to apply this.  You'll need to provide your features matrix (X) and target vector (y) as parameters to the fit method.

```
model.fit(X, y)
```

4. As this is a two dimensional linear model, it has two parameters: The line's intercept and slope. The convention is to add underscores to the names of the parameters it finds.

Look at the coefficients with ```model.estimator_.coef_``` (slope) and ```model.estimator_.intercept_``` (intercept).

5. Take a look at the R^2 value. If more comprehensive numerical statistics are wanted then statsmodel should be used.  We used this briefly last time, and will use it again later today.

Use
```model.score```

6. Make predictions!

But to find the smooth line that seaborn finds we need to explicitly tell scikit-learn that we want to do a prediction for all of the meal costs that we're interested in.  To generate an array of numbers to predict across, use the numpy ```linspace``` (which is short for linear spacing).

Use the ```linspace``` method in numpy like this:

```
x = np.linspace(.5, 5.5, 100)```

to create an array of 100 numbers equally spaced from .5 to 5.5.

```
This is what we need to make predictions but this collection isn't formatted correctly for scikit-learn.  To make it work with scikit-learn we next have to adjust the format with this instruction:

```
xfit = x.reshape(-1,1)
yfit = model.predict(xfit).
```

Lastly, plot the data and model fit. Use matplotlib line and scatter plot.

```
plt.scatter(recovery['reciprocal_feed_sulphur'], recovery['metal_recovery_percent'],label='Data')
plt.plot(xfit, yfit,label='Model Fit')
plt.legend()
```

To set the axes labels, use ```plt.xlabel``` and ```plt.ylabel```. 

## Bonus Exercise

Consider working through these exercises in your spare time.

1. Plot the inlier vs outliers for the training data
2. Re-run the regression using all the data with 0.5 outlier probability (do not split into training and test data).
3. Repeat step 2 a few times. Are the results repeatable?
4. Use the package statsmodel to do regression and output the model fit statistics

Hint: Look at `sm.OLS`

# Exercise 2:  Basis Regression using templates

In multichannel instrumentation, such as XRD and spectroscopy it's common to have hundred or even thousands of spectral channels.  Often there is prior understanding so that we know how pure examples of various components would look like in the instrument.  It's also the case that the behaviour of the samples and the instrument is well-approximated as a linear system.  That is, one where the whole is the sum of the known parts.  The problem then is to find out how much of each known part is present in an observation.

This process is a kind of transform.  We want to transform the data from the original feature space, to a new feature space based on how much of each part is present in the observation.  To make it more concrete, if we're using a spectral instrument, which measures light intensity at every nanometre wavelength between 1000nm and 2000nm, then we have a multichannel instrument with about 1000 channels.  Basis regression says you know what each of the pure components look like to the spectrometer (i.e. a template), and you have reason to think they are additive to make a whole.  The problem then being solved is how much of each of the component parts is present in an observation.  If we have background knowledge that tells us that there could be some combination of three pure components in the observations, we want to find how much of each is present.  This means we turn the original 1000 features into three new features.

In this example of basis regression using templates, we know what the features look like.  PCA, which we will look at after this, doesn't make that assumption.  It can try to learn the component features from the data.  So why would you use basis regression when, superficially, it doesn't seem as clever as PCA?  Sometimes you don't have any training data for PCA, and sometimes you have prior knowledge about what you may see, and when you have this knowledge it can make sense to use it.

In this example we've created templates for four minerals of interest - quartz, dilithium, red kryptonite and unobtainium.  We also have drill hole data containing observations from 50 samples of these components.  Let's use template regression to estimate how much quartz, dilithium, kryptonite and unobtainium are present in each of the observations

1. First use the template_helper module to create the templates for these four phases.  These methods are probably not generally useful to your problems, so there's no need to understand how they work in detail, just that there is code that creates templates.  In a real-world application you might find these in a scientific database.
2. Plot and observe the templates.  Notice that they overlap sometimes.
3. Create a set of synthetic observations, based on these templates and synthetic compositions
4. Is there any background here?  Do you have backgrounds in your data that you need to remove?  If so, discuss how you might do this with your pair programming partner.  You can discuss this at a conceptual level, you don't need to work out an algorithm in this exercise.  Real data often does contain background and often the first task in machine learning processes is to find and remove this.  How this is done is problem dependent, but if you have knowledge in the field that you're applying machine learning to then you'll likely know the common approaches to removing background.  Otherwise you may need to consult a domain or data science expert.  A common approach is to fit a polynomial around a baseline.
5. Linear least-squares fit a solution.
6. Compare the predicted coefficients against the known coefficients.  How did it go?

In [ ]:
from regression_help import MultichannelXAxis

In [ ]:
help(MultichannelXAxis)

In [ ]:
# Use MultichannelXaxis to create x_axis. 
# For the arguments, use 5 for min_x, 90 for max_x, and 0.2 for spacing
x_axis = 

In [ ]:
vars(x_axis)

The following cells show how templates for our synthetic pure components can be constructed by adding gaussians together to form peaks.  While this is a synthetic example, many real-world problems are modelled as gaussians added together - this is often the case in infrared spectroscopy and also with x-ray instrumentation.  These instruments are widely used in energy and minerals.

In [ ]:
# Create a matrix of zeros. We will fill in the required basis for the pure
# components later. 
# There should be one column for each of the pure components, 
# and an additional column so that a constant can be added and modelled. The number
# of rows should be the same as the number of channels in x_axis
templates_matrix = 

In [ ]:
# We should confirm that we have 426 channels for each of the four
# templates, plus one for the constant column.
templates_matrix.shape

In [ ]:
# The as_vector property returns an array with all of the
# channels.  This shows that x_axis.as_vector has 426 elements.
# Each element is a measured feature for each observation.
x_axis.as_vector.shape

In [ ]:
# _apply_convolution_kernels is a bit mysteriously named, but all it does
# is it creates a gaussian peak of a specified width at particular
# positions within the template.  The details aren't too important, just
# know that we're adding together a collection of peaks to make a full
# template for our pure component.
from regression_help import _apply_convolution_kernels

In [ ]:
help(_apply_convolution_kernels)

In [ ]:
# use _apply_convolution_kernels to create a_peak. Use the arguments, 10 for intensity, 
# 25.0 for two_theta_angle, 4 for instrument_broadening_fwhm, 3 for reflection_broadening_fwhm
a_peak = 

In [ ]:
plt.plot(x_axis.as_vector,a_peak)

In [2]:
# Now lets create a full template for quartz.  This problem is 
# written in the language of x-ray diffraction, where peaks are
# called "reflections".  Just think of a reflection as a pointy
# area in the squiggly line.

# This makes a list of the parameters that describe each reflection.
# Then we're going to use _apply_convolution_kernels to make those
# describing parameters real as shapes.
# from collections import namedtuple
quartz_reflections = []
Reflection = namedtuple('Reflection', ('intensity', 'two_theta', 'broadening'))
first = Reflection(intensity = 10.0, two_theta = 25.0, broadening= 3.0)
# The 'append' method adds an item to an existing list.
quartz_reflections.append (first)
quartz_reflections.append (Reflection(13.0, 38.0, 6.0))
quartz_reflections.append (Reflection(10.0, 43.0, 12.0))
quartz_reflections.append (Reflection(25.0, 12.0, 2.0))

In [ ]:
# You should have a list of 4 named tuples, each of which contains parameters
# for the 4 reflections.
quartz_reflections

In [ ]:
first = quartz_reflections[0]
first_reflection_of_quartz = _apply_convolution_kernels(x_axis.as_vector, first.intensity, 
                                                        first.two_theta, 4, first.broadening)

In [ ]:
# Here's what the first "reflection" (or peak) looks like.
plt.plot(x_axis.as_vector, first_reflection_of_quartz)

In [ ]:
# Now add it to the templates matrix.  Have you seen the += notation before?
# It means that we add the right hand side to whatever we had before
# on the left hand side and save the result back to the same place.
templates_matrix[:, 0] += first_reflection_of_quartz

In [ ]:
# Lets confirm that this worked properly by plotting the result.  It
# should look the same as first_reflection_of_quartz.
# Use plt.plot(x, y)
plt.

In [ ]:
# Lets keep going with the remaining reflections.
second = 
second_reflection_of_quartz = _apply_convolution_kernels(x_axis.as_vector, second.intensity, 
                                                        second.two_theta, 4, second.broadening)

In [ ]:
plt.plot(x_axis.as_vector, second_reflection_of_quartz)

In [ ]:
# Notice how the += operator adds together the two peaks to make a 
# composite shape.
templates_matrix[:, 0] += second_reflection_of_quartz

In [ ]:
plt.plot(x_axis.as_vector, templates_matrix[:, 0])

In [ ]:
# Now that you have the gist of how the templates are being created, lets
# just use the create_template_matrix method which will create all of our
# templates this way.
from regression_help import create_templates_matrix

In [ ]:
templates = 

In [ ]:
# here's the final quartz template
plt.plot(x_axis.as_vector, templates[:, 0],label='quartz')
plt.legend()

In [ ]:
# dilithium
plt.plot(x_axis.as_vector, )
plt.legend()

In [ ]:
# kryptonite
plt.plot(x_axis.as_vector, )
plt.legend()

In [ ]:
# unobtainium
plt.plot(x_axis.as_vector, )
plt.legend()

In [ ]:
# Now we have pure components as they might be seen by our
# instrument.  To test our template regression method we
# need to make up some compositions and then multiply our pure
# patterns by these amounts to make the final observed
# patterns.  The underlying assumption is that the observations
# will be a "linear combination" of the parts.

from regression_help import create_composition_dataframe

In [ ]:
# Make 50 compositions.  The dataframe we're making can be thought
# of as the ground truth, or gold standard assays for each pure
# component. These are essentially the target values we would like to model.
compositions_dataframe = 

In [ ]:
compositions_dataframe.head()

In [ ]:
# Now we can use create_observations to sum the templates
# in proportion to how much of each pure component is present
# as specified in the compositions_dataframe.  This gives us
# our synthetic observations.
from regression_help import create_observations

In [2]:
# But first, like with the template generation, let's manually create 
# some observations so that we understand how the
# code in create_observations works.  

# Let's make a single observation before using create_observations to create 
# a set of them for us.

# We sum together the contribution of each phase to create an 
# observation in a similar way to how we summed together peaks
# to make the pure templates.  See if you can follow how
# the arrays are being manipulated below.

plt.plot(x_axis.as_vector,
         templates[:,0]*compositions_dataframe['Quartz'][1], label='template*quartz[1]')
plt.plot(x_axis.as_vector,
         templates[:,0],  label='template')
plt.legend()

In [ ]:
# Get the observation due to the Quartz component, using only the second row of the 
# compositions_dataframe
observation = 
observation

In [ ]:
# How many observations do we have? Check compositions_dataframe.
observations_count = 
observations_count

In [ ]:
# How many channels are there in template?
channels_count = 
channels_count

In [ ]:
# Create an empty matrix (filled with zeros) with number of channels 
# as number of rows and number of observations as number of columns
observations_matrix = 
observations_matrix.shape

In [ ]:
observations_matrix[:, 1]= 

In [ ]:
plt.plot(x_axis.as_vector, 
         observations_matrix[:, 1])

In [ ]:
# Now add the component due to Dilithium
observations_matrix[:, 1] = observations_matrix[:, 1] + templates[:,1]*compositions_dataframe['Dilithium'][1]

# And kryptonite
observations_matrix[:, 1] = observations_matrix[:, 1] + templates[:,2]*compositions_dataframe['Kryptonite'][1]

# And unobtainium
observations_matrix[:, 1] = observations_matrix[:, 1] + templates[:,3]*compositions_dataframe['Unobtainium'][1]

In [ ]:
plt.plot(x_axis.as_vector, 
         observations_matrix[:, 1])

Now that we have seen how an observation is created, use create_observations to generate observations.

In [ ]:
observations_matrix = 

In [ ]:
# Let's see the third observation (they start counting from zero). 
# See that the create_observations function also includes the addition of gaussian noise 
plt.plot(x_axis.as_vector, observations_matrix[:,2])

In [ ]:
# Let's see the first observation.  Look closely, there is a difference!
plt.plot(x_axis.as_vector, observations_matrix[:,0], label = 'Observation 1')
plt.plot(x_axis.as_vector, observations_matrix[:,2], label = 'Observation 3')
plt.legend()

In [ ]:
plt.figure()
[plt.plot(x_axis.as_vector, observations_matrix[:,i], label = 'Observation ' + str(i)) for i in np.arange(observations_count)]
plt.show()

Now set up a linear regression to solve for how much of each of our pure components is present in our observations.  This problem is a little bit different to our previous linear regressions in a couple of ways.

The meaning of features is more complex and maybe less intuitive than before.  This is a kind of transform, one that takes us from how much of one set of features is present (the amounts of the two-theta channels) to how much of another set of features is present (the amounts of the templates). So features have a dual meaning here.

Our instrument gives us 426 features as "two-theta" channels.  Each channel is like a single "x" number, like with feed_sulphur in the earlier example, but we have 426 of them now.  More interestingly, we want to use linear regression to express each observation in terms of how much of the template arrays are present.  There are 4 of these (plus a template of ones, that is needed for the maths book keeping to work properly).  So our features are now not a single number, but a matrix of templates.  Previously we used the [[]] notation to create a matrix out of the single x variable, but now sklearn is in its element and we can use the templates array as it is, as the feature matrix.

What the features are depends on what the question is that we're asking linear regression to solve for.  In some problems we have features, and we have the observed "amount" of each feature present.  This is what we get from our instrument, where each feature is a two-theta.  But the question we want help with is how much of our new template features are present in our observations.

The target array is also less intuitive in this case.  To re-iterate, the problem we're setting up is "How much of each feature is present in an observation?".  A feature in this case is a template.  So the target array is the instrument observations for each of the 426 channels.  The instrument observation now appears in the target array, instead of the features array, like it did with predicting metal recovery from sulphur.

Sometimes, to use machine learning to solve a problem we need to re-formulate the question in a way that makes it more tractable.  Often figuring out an appropriate set of features is a first, and sometimes difficult, step that can make a big difference to the results.

Use scikit-learn to solve this problem, and solve for the first observation.  It'll only take a few lines, once you've gone through the mental gymnastics to get to it!

The intent of manually playing with the templates and the observations was to help you build up a feel for what the data looks like that you're now trying to make a model for.

In [ ]:
#from sklearn.linear_model import LinearRegression
X = 
y = 
model = 
model.fit(X, y)

In [ ]:
# use model.coef_ to get the calculated linear regression coefficients


How do these estimates for our phases compare with the ground truth? (Look for ground truth in ```compositions_dataframe```)

Not bad!

## Bonus

1. Use the stastmodel.api to fit a model to the first observation data frame

In [ ]:
# Bonus 2 would look something like this
endog = 
exog = 
model = 
results = model.fit()
results.summary()

#### Regularisation

Often your real world problem will have thousands of templates.  Linear least-squares has a habit of finding non-zero amounts of many of the templates if in a database.  This is because the least-squares criteria doesn't concern itself with how many of the templates it finds in an observation.

Why does that matter?  Often, when an instrument sees a sample, we know that the sample will only ever contain at most a dozen pure components - it may be the nature of how the small spatial volume that the sample came from, and that there are only so many pure components that are ever likely to be found in such a small space.  But noisy instrument data and operator error, together with the least-squares optimisation goal, can mean that we end up with lots of unreal false-positives for components in our observations.

In this situation we often prefer what are called regularised solvers.  A particular class of regularised solvers, called L1 regularised solvers, have a built-in preference for only seeing a small number of components.

Scikit-learn makes these as easy to do as least squares linear regression.  You should look into this if it may be relevant to your industrial problems.  An example of an L1 regularised solver is "LASSO".

#### What next?

Now that we've transformed our data according to a new, known, lower dimensional "basis", we're ready to tackle the problem of finding an underlying basis when we don't have any templates, but we do have data that we can use for training.  The most well-known and fundemental method for this is Principal Components Analysis (PCA).